In [1]:
# based on https://github.com/chkwon/TrafficAssignment.jl

In [2]:
include("../Julia_files/load_network_uni-class.jl")

load_ta_network (generic function with 2 methods)

In [3]:
using Graphs

function create_graph(start_node, end_node)
    @assert Base.length(start_node)==Base.length(end_node)

    no_node = max(maximum(start_node), maximum(end_node))
    no_arc = Base.length(start_node)

    graph = simple_inclist(no_node)
    for i=1:no_arc
        add_edge!(graph, start_node[i], end_node[i])
    end
    return graph
end

create_graph (generic function with 1 method)

In [4]:
function get_vector(state, origin, destination, link_dic)
    current = destination
    parent = -1
    x = zeros(Int, maximum(link_dic))

    while parent != origin
        parent = state.parents[current]

        link_idx = link_dic[parent,current]

        if link_idx != 0
            x[link_idx] = 1
        end

        current = parent
    end

    return x
end

get_vector (generic function with 1 method)

In [5]:
ta_data = load_ta_network("Sioux Falls")

TA_Data("Sioux Falls",24,24,1,76,[1,1,2,2,3,3,3,4,4,4  …  22,22,22,22,23,23,23,24,24,24],[2,3,1,6,1,4,12,3,5,11  …  15,20,21,23,14,22,24,13,21,23],[25900.2,23403.5,25900.2,4958.18,23403.5,17110.5,23403.5,17110.5,17782.8,4908.83  …  9599.18,5075.7,5229.91,5000.0,4924.79,5000.0,5078.51,5091.26,4885.36,5078.51],[6.0,4.0,6.0,5.0,4.0,4.0,4.0,4.0,2.0,6.0  …  3.0,5.0,2.0,4.0,4.0,4.0,2.0,4.0,3.0,2.0],[6.0,4.0,6.0,5.0,4.0,4.0,4.0,4.0,2.0,6.0  …  3.0,5.0,2.0,4.0,4.0,4.0,2.0,4.0,3.0,2.0],[0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15  …  0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15],[4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0  …  4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0],[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0  …  0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0  …  0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],[1,1,1,1,1,1,1,1,1,1  …  1,1,1,1,1,1,1,1,1,1],360600.0,24x24 Array{Float64,2}:
    0.0  100.0  100.0   500.0   200.0  …   100.0   400.0   300.0   100.0
  

In [6]:
ta_data.od_pairs

576-element Array{(Int64,Int64),1}:
 (1,1)  
 (1,2)  
 (1,3)  
 (1,4)  
 (1,5)  
 (1,6)  
 (1,7)  
 (1,8)  
 (1,9)  
 (1,10) 
 (1,11) 
 (1,12) 
 (1,13) 
 ⋮      
 (24,13)
 (24,14)
 (24,15)
 (24,16)
 (24,17)
 (24,18)
 (24,19)
 (24,20)
 (24,21)
 (24,22)
 (24,23)
 (24,24)

In [7]:
# unpacking data from ta_data
network_name = ta_data.network_name

number_of_zones = ta_data.number_of_zones
number_of_nodes = ta_data.number_of_nodes
first_thru_node = ta_data.first_thru_node
number_of_links = ta_data.number_of_links

start_node = ta_data.start_node
end_node = ta_data.end_node
capacity = ta_data.capacity
link_length = ta_data.link_length

free_flow_time = ta_data.free_flow_time
B = ta_data.B
power = ta_data.power
speed_limit = ta_data.speed_limit
toll = ta_data.toll
link_type = ta_data.link_type
number_of_zones = ta_data.number_of_zones
total_od_flow = ta_data.total_od_flow
travel_demand = ta_data.travel_demand
od_pairs = ta_data.od_pairs

toll_factor = ta_data.toll_factor
distance_factor = ta_data.distance_factor

best_objective = ta_data.best_objective

0.0

In [8]:
# dividing the travel_demand into two classes (car and truck)
travel_demand_car = travel_demand * .8
travel_demand_truck = travel_demand * .2

assert(travel_demand_car + travel_demand_truck == travel_demand)

In [9]:
# preparing a graph
graph = create_graph(start_node, end_node)
link_dic = sparse(start_node, end_node, 1:number_of_links)

24x24 sparse matrix with 76 Int64 entries:
	[2 ,  1]  =  3
	[3 ,  1]  =  5
	[1 ,  2]  =  1
	[6 ,  2]  =  14
	[1 ,  3]  =  2
	[4 ,  3]  =  8
	[12,  3]  =  35
	[3 ,  4]  =  6
	[5 ,  4]  =  11
	[11,  4]  =  31
	⋮
	[24, 21]  =  75
	[15, 22]  =  46
	[20, 22]  =  63
	[21, 22]  =  65
	[23, 22]  =  72
	[14, 23]  =  42
	[22, 23]  =  70
	[24, 23]  =  76
	[13, 24]  =  39
	[21, 24]  =  66
	[23, 24]  =  73

In [10]:
graph.vertices

1:24

In [11]:
function BPR(x)
    # travel_time = free_flow_time .* ( 1.0 + B .* (x./capacity).^power )
    # generalized_cost = travel_time + toll_factor *toll + distance_factor * link_length
    # return generalized_cost

    bpr = similar(x)
    for i=1:length(bpr)
        bpr[i] = free_flow_time[i] * ( 1.0 + B[i] * (x[i]/capacity[i])^power[i] )
    end
    return bpr
end

BPR (generic function with 1 method)

In [12]:
function all_or_nothing(travel_time)
    state = []
    path = []
    x = zeros(size(start_node))

    for r=1:size(travel_demand)[1]
        # for each origin node r, find shortest paths to all destination nodes
        state = dijkstra_shortest_paths(graph, travel_time, r)

        for s=1:size(travel_demand)[2]
            # for each destination node s, find the shortest-path vector
            # load travel demand
            x = x + travel_demand[r,s] * get_vector(state, r, s, link_dic)
        end
    end

    return x
end

all_or_nothing (generic function with 1 method)

In [13]:
function all_or_nothing_car(travel_time_car)
    state = []
    path = []
    x = zeros(size(start_node))

    for r=1:size(travel_demand_car)[1]
        # for each origin node r, find shortest paths to all destination nodes
        state = dijkstra_shortest_paths(graph, travel_time_car, r)

        for s=1:size(travel_demand_car)[2]
            # for each destination node s, find the shortest-path vector
            # load travel demand
            x = x + travel_demand_car[r,s] * get_vector(state, r, s, link_dic)
        end
    end

    return x
end

all_or_nothing_car (generic function with 1 method)

In [14]:
function all_or_nothing_truck(travel_time_truck)
    state = []
    path = []
    x = zeros(size(start_node))

    for r=1:size(travel_demand_truck)[1]
        # for each origin node r, find shortest paths to all destination nodes
        state = dijkstra_shortest_paths(graph, travel_time_truck, r)

        for s=1:size(travel_demand_truck)[2]
            # for each destination node s, find the shortest-path vector
            # load travel demand
            x = x + travel_demand_truck[r,s] * get_vector(state, r, s, link_dic)
        end
    end

    return x
end

all_or_nothing_truck (generic function with 1 method)

In [15]:
# Finding a starting feasible solution
travel_time = BPR(zeros(number_of_links));
xl = all_or_nothing(travel_time);

travel_time_car = 1.0 * BPR(zeros(number_of_links));
travel_time_truck = 1.1 * BPR(zeros(number_of_links));
#travel_time_truck = 1.0 * BPR(zeros(number_of_links));

xl_car = all_or_nothing_car(travel_time_car);
xl_truck = all_or_nothing_truck(travel_time_truck);

In [16]:
max_iter_no = 1e3
l = 1
#average_excess_cost = 1
tol = 1e-5

while l < max_iter_no
    l += 1
    
    ## Finding yl
    #travel_time = BPR(xl)
    #yl = all_or_nothing(travel_time)
    #xl = xl + (yl - xl)/l
    
    # Finding yl_car
    xl_car_old = xl_car
    travel_time_car = 1.0 * BPR(xl_car + 2.0 * xl_truck)
    yl_car = all_or_nothing_car(travel_time_car)
    xl_car = xl_car + (yl_car - xl_car)/l
    xl_car_new = xl_car
    
    # Finding yl_truck
    xl_truck_old = xl_truck
    travel_time_truck = 1.1 * BPR(xl_car + 2.0 * xl_truck)
    yl_truck = all_or_nothing_truck(travel_time_truck)
    xl_truck = xl_truck + (yl_truck - xl_truck)/l
    xl_truck_new = xl_truck
    
    
    relative_gap_car = norm(xl_car_new - xl_car_old, 1) / norm(xl_car_new, 1)
    relative_gap_truck = norm(xl_truck_new - xl_truck_old, 1) / norm(xl_truck_new, 1)
    
    if l % 10000 == 0
        print("l = $l------------------------------------------------\n")
        print("relative_gap_car is $relative_gap_car\n")
        print("relative_gap_truck is $relative_gap_truck\n")
    end

    if relative_gap_car < tol && relative_gap_truck < tol
        print("l = $l------------------------------------------------\n")
        print("relative_gap_car is $relative_gap_car\n")
        print("relative_gap_truck is $relative_gap_truck\n")
        break
    end
    
end

In [17]:
outfile = open("../data_original/flows_car_converge_1e-5.txt", "w")

write(outfile, join(("From", "to", "Volume_Capacity"), "        "), "\n")

for i = 1:length(ta_data.start_node)
     n1, n2, n3 = ta_data.start_node[i], ta_data.end_node[i], xl_car[i]
     write(outfile, join((n1, n2, n3), "        "), "\n")
end

close(outfile)

In [18]:
outfile = open("../data_original/flows_truck_converge_1e-5.txt", "w")

write(outfile, join(("From", "to", "Volume_Capacity"), "        "), "\n")

for i = 1:length(ta_data.start_node)
     n1, n2, n3 = ta_data.start_node[i], ta_data.end_node[i], xl_truck[i]
     write(outfile, join((n1, n2, n3), "        "), "\n")
end

close(outfile)